# Universidade Federal de Campina Grande 
# Unidade acadêmica de Sistemas e Computação - Pós-graduação 
# Professor: Leandro Balby
# Machine Learning 2017.2
# Aluno:  Caio Batista Oliveira 

# ----------------------------------------------------------------------------------------------------------

# Classificação de violações de design em programas de alunos utilizando  redes neurais

## 1) Introdução 

O tema do meu mestrado tem como princípio testes de identificação de violações de padrão de design para programas dos alunos da disciplina de <b>Programação 1</b>. E a ideia inicial deste projeto para a disciplina de Machine Learnin era fazer uma classificação de um programa conforme sua unidade da disciplina, entretanto uma ideia que fazia mais sentido era mudar o foco para a detecção de violação de programas de uma forma baseada em dados. Foi então que mudei o projeto para fazer uma classificação especificamente entre <b>Violação</b> e <b>Não-violação</b> de programas especificamente da unidade 7. A escolha dessa unidade foi dada por ter uma grande margem para utilização de funções e operações sobre listas, o problema mais recorrente na pesquisa do mestrado. 

A questão da classificação fica mais fácil de ser compreendida com exeplos, então a seguir temos a lista de funções proibidas e logo abaixo 3 exemplos e suas respectivas classificações com uma curta explicação.

+ replace
+ count 
+ index
+ sort 
+ min 
+ max
+ join
+ sum
+ strip

1 Exemplo - Neste exemplo não existe violação, a iteração sobre a lista é permitida e o acumulo na variável também. 

In [213]:
# Não-violação 

def exemplo1(lista):
    soma = 0
    for i in range(len(lista)):
        soma += lista[i]

    return soma    
        

2 Exemplo - Neste exemplo temos uma violação na ordenação da lista, pois a operação <b>Sort</b> não é permitida.

In [214]:
# Violação 

def exemplo2(lista):
    return lista.sort()
        

3 Exemplo - Aqui neste último exemplo temos uma violação. Apesar da operação <b>Split</b> ser permitida, o <b>Join</b> não é.

In [215]:
# Violação 

def exemplo3(string):
    entrada = string.split()
    entrada = "-#-".join(entrada)
   
    return entrada

<p></p>

## 2) Categorização, filtragem e processamento dos dados

A principal dificuldade em relação a esse projeto foram os dados. Apesar de existir um considerável dataset de programas de alunos da disciplina de Programação 1, esses dados são de propriedade dos alunos e dos professores então nesse projeto não posso divulgar nenhum dos dados de treino e de teste. Entretanto todos eles vieram dessas amostras dos alunos de forma anônima. Mais sobre o a quantidade detalhada de cada parte dos dados (treino, teste e validação) em tópicos mais a frente. 

Para a filtragem dos dados foram utilizadas duas técnicas, uma utilizando a ferramenta que estou desenvolvendo na pesquisa do mestrado e uma segunda etapa de forma manual em cada uma das amostras. 

O único processamento feito nos dados foi um parser para ler o código dentro de cada módulo Python e transforma-lo em uma string para a rede neural.

## 3) Definição da Rede Neural e arquitetura


Como este problema involve diretamente leitura de código e <b>tokenization</b> optei por utilizar uma ferramenta que ajuda bastante nessa questão e funcionou bem com a construção da rede, chamada <b>NLTK</b>(natural language tool kit). Essa ferramenta se estende a diversos idiomas, como a linguagem Python é basicamente escrita em inglês e geralmente as variáveis dos programas estão em inglês ou português esse kit se ajustou bem a necessidade do projeto. 

Antes de definir a Rede Neural propriamente, vale ressaltar que a ideia desse projeto foi baseada na classificação de textos utilizando uma ANN, como foi passado em sala de aula. Nesse caso tratando o código como um comentário que vai ter suas palavras-chave aprendidas e atribuidas pesos para depois determinar se ela se encaixa em determinada classificação ou não. 

<p></p>

Falaremos agora aqui sobre a arquitetura da rede neural em sí. A ANN (artificial neural network) que foi construida é composta de duas camadas sendo uma delas escondida, e ainda um BOW (bag of words) para ajudar no treinamento da rede. E tem como sua estrutura a imagem que se segue.

ANN 2-layer (1 hidden): 
![alt text](http://cs231n.github.io/assets/nn1/neural_net.jpeg "Rede neural")


Quanto a seu funcionamento, nas duas camadas foram utilizadas a função <b>Sigmoid</b> para as duas sinapses. Que foram iniciadas com valores aleatórios com média 0, mas que seriam atualizados pelo algoritmo <b>Gradiente Descendente</b> visto em sala, que tem como objetivo minimizar a função de erro. E  basicamente o que ela faz é a cada iteração dar um peso para cada palavra do bow baseado na presença delas na sentença dada (input).

## 4) Construção da Rede Neural e experimento


Em primeiro lugar temos que fazer o importar as bibliotecas que usaremos. <b>NLTK</b> como já foi falado para a manipulação da linguagem e criação de tokens. O <b>json</b> pois as sinapses ficaram em formato json para serem armazenadas. E finalmente <b>glob</b> para manipular arquivos e filtragem. 

In [216]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import os
import json
import glob
import datetime
import random
from random import shuffle

stemmer = LancasterStemmer()

Foram criadas duas funções auxiliares justamente para essa manipulação de arquivos e processamento do código dos programas que são dados para essa rede neural, retirando aquelas informações desnecessárias nos programas como espaçamento, virgulas e pontos. 

In [217]:
def get_data_from_files(dir):
    list_data = []
    files = get_files_names(dir)
    for f in files:
        with open(f, 'r') as myfile:
            data=myfile.read().replace('\n', ' ')
            data=data.replace('.',' ')
            data=data.replace('\t',' ')
            data=data.replace('(',' ')
            data=data.replace(')',' ')
            
            data=(" ".join(data.split()))
        list_data.append(data)
        
    
    return list_data


def get_files_names(dir):
    files = glob.glob(dir + '/*.py')
    return files

    

Aqui temos a divisão dos dados em teste e treino. Além da criação das duas classes em questão, nesse caso <b>violation</b> e <b>notviolation</b>. Totalizando 405 no total de dados de treino e 81 para testes.

In [218]:
not_violations = get_data_from_files('not_violations')
violations = get_data_from_files('violations')
test_files = get_data_from_files('test')

training_data = []
for e in not_violations:
    training_data.append({"class":"notviolation", "sentence":e})

for e in violations:
    training_data.append({"class":"violation", "sentence":e})


print ("%s sentences clean of violation for training data" % len(not_violations))
print ("%s sentences with violation for training data" % len(violations))
print ("%s sentences in total training data" % len(training_data))
print ("%s sentences in total test data" % len(test_files))


270 sentences clean of violation for training data
135 sentences with violation for training data
405 sentences in total training data
81 sentences in total test data


<p></p>

Começamos então a criação do BOW aqui denominado <b>words</b>, sempre tratando todas as palavras com letras minúsculas e criando seus tokens. Além disso existem uma relação de palavras que caso apareçam serão ignoradas, são elas pontuações da linguagem. <b>Documents</b> aqui trata da relação de presença do token no BOW. No final são removidas as duplicadas através de um <b>set</b>.

In [219]:
words = []
classes = []
documents = []
ignore_words = ['?',':',',','[',']']

for pattern in training_data:
    w = nltk.word_tokenize(pattern['sentence'])
    words.extend(w)
    documents.append((w, pattern['class']))
    if pattern['class'] not in classes:
        classes.append(pattern['class'])

words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]

words = list(set(words))
classes = list(set(classes))

print (len(documents), "documents")
print (len(classes), "classes", classes)

405 documents
2 classes ['notviolation', 'violation']


<p></p>

Nesta parte organiza-se os dados e conclui a criação do BOW com os <b>documents</b> criados antes. Totalizando 651 palavras no BOW  e as duas classes que esperamos classificar.

In [220]:
training = []
output = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    training.append(bag)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    output.append(output_row)

print ("words: ", len(words))
print ("classes: ", len(classes))

words:  651
classes:  2


<p></p>

Com o BOW concluido um exemplo de como os dados estão organizados é mostrado abaixo. Primeiramente temos o programa em forma de lista de tokens, depois disso o BOW de 0's e 1's para determinar se aquela palavra que está no BOW também está naquela sentença. E por último a qual classe esse programa pertence.

In [221]:
i = 0
words_sample = documents[i][0]
token_words = [stemmer.stem(word.lower()) for word in words_sample]
shuffle(token_words)
print (token_words)
print (training[i])
print (output[i])

['-1', 'in', 'in', ']', 'len', 'l1', ':', 'def', 'j', 'j+1', 'return', 'l1', '=', 'j', '[', 'troc', 'l1', '=', 'for', 'j', 'l1', ':', 'rang', '[', ']', 'insere_ordenado_primeiro', '[', ']', 'l1', 'l1', 'j', ']', 'l1', '[', ']', ':', 'for', 'troc', 'j+1', '=', '[', 'if', 'rang', 'l1', 'l1', 'j+1', 'len', ']', '>', '[', 'i', ':', 'l1']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

<p></p>

Nesta parte definimos funções extremamentes importantes para os cálculos a cada iteração da rede. Primeiro o importando o <b>numpy</b> que pode fazer operações sobre matrizes de uma forma bem mais eficiente e nos poupa tempo de implementar funções que podem ser black-box. Depois temos a função de ativação que é a mesma para as duas camadas: <b>Sigmoid</b>. A derivada dela também está presente aqui para calcularmos o erro da rede. 

As duas últimas funções são bem importantes pois o <b>bow</b> gera a lista de 0's e 1's que vimos anteriormente no exemplo para um dado programa (tratado como sentença aqui). Já a função <b>think</b> faz o cálculo das sinapses com as camadas.  

In [222]:
import numpy as np
import time

def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

def sigmoid_output_to_derivative(output):
    return output*(1-output)
 
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1

    return(np.array(bag))

def think(sentence):
    x = bow(sentence.lower(), words)
    l0 = x
    l1 = sigmoid(np.dot(l0, synapse_0))
    l2 = sigmoid(np.dot(l1, synapse_1))
    
    return l2


O treinamento utiliza um algoritmo de <b>Gradiente Descendente</b> que foi visto em sala, apenas com algumas alterações para o funcionamento com o BOW e as funções de ativação.  

Nessa parte especificamente podemos resumir o algoritmo como: Inicialização dos pesos das sinapses de forma aleatoria, para cada iteração calcula-se os valores das camadas e os erros delas em sequencia, se o erro for maior que o da iteração anterior o algoritmo para, se não ele continua até o final das iterações. Atualizando sempre ao final os pesos da sinapses e o gradiente.    

No final de tudo as sinapses são armazenadas em um arquivo no formato <b>json</b>.

In [223]:
def train(X, y, hidden_neurons=10, alpha=1, iterations=50000):

    print ("Training with %s neurons, alpha:%s " % (hidden_neurons, str(alpha)) )
    print ("Input matrix: %sx%s    Output matrix: %sx%s" % (len(X),len(X[0]),1, len(classes)) )
    np.random.seed(1)

    last_mean_error = 1
    synapse_0 = 2*np.random.random((len(X[0]), hidden_neurons)) - 1
    synapse_1 = 2*np.random.random((hidden_neurons, len(classes))) - 1

    prev_synapse_0_weight_update = np.zeros_like(synapse_0)
    prev_synapse_1_weight_update = np.zeros_like(synapse_1)

    synapse_0_direction_count = np.zeros_like(synapse_0)
    synapse_1_direction_count = np.zeros_like(synapse_1)
        
    for j in iter(range(iterations+1)):
        layer_0 = X
        layer_1 = sigmoid(np.dot(layer_0, synapse_0))
        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        layer_2_error = y - layer_2

        if (j% 10000) == 0 and j > 5000:
            if np.mean(np.abs(layer_2_error)) < last_mean_error:
                print ("delta after "+str(j)+" iterations:" + str(np.mean(np.abs(layer_2_error))) )
                last_mean_error = np.mean(np.abs(layer_2_error))
            else:
                print ("break:", np.mean(np.abs(layer_2_error)), ">", last_mean_error )
                break
                

        layer_2_delta = layer_2_error * sigmoid_output_to_derivative(layer_2)
        layer_1_error = layer_2_delta.dot(synapse_1.T)
        layer_1_delta = layer_1_error * sigmoid_output_to_derivative(layer_1)
        
        
        synapse_1_weight_update = (layer_1.T.dot(layer_2_delta))
        synapse_0_weight_update = (layer_0.T.dot(layer_1_delta))
        
        if(j > 0):
            synapse_0_direction_count += np.abs(((synapse_0_weight_update > 0)+0) - ((prev_synapse_0_weight_update > 0) + 0))
            synapse_1_direction_count += np.abs(((synapse_1_weight_update > 0)+0) - ((prev_synapse_1_weight_update > 0) + 0))        
        
        synapse_1 += alpha * synapse_1_weight_update
        synapse_0 += alpha * synapse_0_weight_update
        
        prev_synapse_0_weight_update = synapse_0_weight_update
        prev_synapse_1_weight_update = synapse_1_weight_update

    now = datetime.datetime.now()

    synapse = {'synapse0': synapse_0.tolist(), 'synapse1': synapse_1.tolist(),
               'datetime': now.strftime("%Y-%m-%d %H:%M"),
               'words': words,
               'classes': classes
              }
    synapse_file = "synapses.json"

    with open(synapse_file, 'w') as outfile:
        json.dump(synapse, outfile, indent=4, sort_keys=True)
    print ("saved synapses to:", synapse_file)

## 5) Tunando a rede


A função de predição aqui foi chamada de <b>classify</b> e tem como principio receber uma sentença, que é um programa depois do parser, e aplicar a função  <b>think</b> e retornar sua classificação de forma ordenada.

Dos parâmetros de treino temos <b>hidden_neurons, alpha</b> e <b>iterations</b> que podem ser tunados. Foram utilizadas 3 diferentes configurações para a análise dos resultados. No final de cada um dos diferentes treinos é mostrada a acurácia do modelo.

Também se tem uma condição de ordenação chamada <b>ERROR_THRESHOLD</b> na qual se for maior que 20% a diferença entre as classes só se imprime a probabilidade da maior.

In [244]:
def classify(sentence):
    results = think(sentence)

    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[classes[r[0]],r[1]] for r in results]
    print ("classification: %s" % (return_results))
    return return_results

test = get_data_from_files('test')
test_names = get_files_names('test')

### 1 Configuração - hidden_neurons=10, alpha=1, iterations=50000  

In [258]:

X = np.array(training)
y = np.array(output)

start_time = time.time()

train(X, y, hidden_neurons=10, alpha=1, iterations=50000)

# probability threshold
ERROR_THRESHOLD = 0.2
# load our calculated synapse values
synapse_file = 'synapses.json' 
with open(synapse_file) as data_file: 
    synapse = json.load(data_file) 
    synapse_0 = np.asarray(synapse['synapse0']) 
    synapse_1 = np.asarray(synapse['synapse1'])


elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

Training with 10 neurons, alpha:1 
Input matrix: 405x651    Output matrix: 1x2
delta after 10000 iterations:0.333333332917
delta after 20000 iterations:0.333333332817
delta after 30000 iterations:0.333333332657
delta after 40000 iterations:0.333333332369
delta after 50000 iterations:0.333333331762
saved synapses to: synapses.json
processing time: 146.9043002128601 seconds


In [259]:
accuracy = 0

for i in range(len(test)):
    print(test_names[i])
    result = classify(test[i])
    
    if ('violation' in test_names[i] and result[0][0] == 'violation') or \
     ('violation' not in test_names[i] and result[0][0] == 'notviolation'):
        accuracy += 1.0
    
    print() 
    
 

test/733e522cba6c8d3fc79cd32546fadf54.py
classification: [['notviolation', 0.9999999999999909]]

test/04aa2288beeb67289e9d5edaa9c10d41.py
classification: [['notviolation', 0.99999999999996581]]

test/57327a51d2fb7ab381497f8420b71411.py
classification: [['notviolation', 0.99999999999936562]]

test/82f2ac914f72771bbfe68ab70ab8c0d1.py
classification: [['notviolation', 0.99999999999984879]]

test/3757d0261bdeddbc4b79aa52169b7bce.py
classification: [['notviolation', 0.99999999999698219]]

test/1927ee1c31d541abeae56ec1a2e0fadf.py
classification: [['notviolation', 0.99999999993401101]]

test/e72f92da1eaa50345ffea0899f9ebbe5.py
classification: [['notviolation', 0.9999999999724174]]

test/de97abfb65d2b5327965ab99089f56e4.py
classification: [['notviolation', 0.99999999999400369]]

test/violation_46.py
classification: [['notviolation', 0.99999999935359751]]

test/violation_33.py
classification: [['notviolation', 0.99999999985068344]]

test/90036ab09a7539f7dff743e33c6e24cf.py
classification: [['no

In [260]:
print("Accuracy: %s%%" %(accuracy/len(test) * 100))   

Accuracy: 70.37037037037037%


<p></p>

### 2 Configuração - hidden_neurons=15, alpha=0.1, iterations=100000  

In [261]:


X = np.array(training)
y = np.array(output)

start_time = time.time()
train(X, y, hidden_neurons=15, alpha=0.1, iterations=100000)

# probability threshold
ERROR_THRESHOLD = 0.2
# load our calculated synapse values
synapse_file = 'synapses.json' 
with open(synapse_file) as data_file: 
    synapse = json.load(data_file) 
    synapse_0 = np.asarray(synapse['synapse0']) 
    synapse_1 = np.asarray(synapse['synapse1'])

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

Training with 15 neurons, alpha:0.1 
Input matrix: 405x651    Output matrix: 1x2
delta after 10000 iterations:0.00372541230022
delta after 20000 iterations:0.00332812946533
delta after 30000 iterations:0.00315700676935
delta after 40000 iterations:0.00305664947844
delta after 50000 iterations:0.00298903141037
delta after 60000 iterations:0.00293970339687
delta after 70000 iterations:0.00290182789618
delta after 80000 iterations:0.00287170164791
delta after 90000 iterations:0.002847132488
delta after 100000 iterations:0.00282674750057
saved synapses to: synapses.json
processing time: 339.45823335647583 seconds


In [262]:
accuracy = 0

for i in range(len(test)):
    print(test_names[i])
    result = classify(test[i])
    
    if ('violation' in test_names[i] and result[0][0] == 'violation') or \
     ('violation' not in test_names[i] and result[0][0] == 'notviolation'):
        accuracy += 1.0
    
    print() 
    
 

test/733e522cba6c8d3fc79cd32546fadf54.py
classification: [['notviolation', 0.99999994681454385]]

test/04aa2288beeb67289e9d5edaa9c10d41.py
classification: [['notviolation', 0.99943335762820074]]

test/57327a51d2fb7ab381497f8420b71411.py
classification: [['notviolation', 0.99898651805160643]]

test/82f2ac914f72771bbfe68ab70ab8c0d1.py
classification: [['notviolation', 0.99974351535192107]]

test/3757d0261bdeddbc4b79aa52169b7bce.py
classification: [['notviolation', 0.98429955334866193]]

test/1927ee1c31d541abeae56ec1a2e0fadf.py
classification: [['notviolation', 0.99919815305133086]]

test/e72f92da1eaa50345ffea0899f9ebbe5.py
classification: [['notviolation', 0.99999973751478011]]

test/de97abfb65d2b5327965ab99089f56e4.py
classification: [['notviolation', 0.99999984746624127]]

test/violation_46.py
classification: [['violation', 0.99999585619601428]]

test/violation_33.py
classification: [['violation', 0.99991849823663637]]

test/90036ab09a7539f7dff743e33c6e24cf.py
classification: [['notvio

In [263]:
print("Accuracy: %s%%" %(accuracy/len(test) * 100))   

Accuracy: 90.12345679012346%


<p></p>

### 3 Configuração - hidden_neurons=25, alpha=0.01, iterations=100000 

In [241]:

    
X = np.array(training)
y = np.array(output)

start_time = time.time()

train(X, y, hidden_neurons=25, alpha=0.01, iterations=100000)

# probability threshold
ERROR_THRESHOLD = 0.2
# load our calculated synapse values
synapse_file = 'synapses.json' 
with open(synapse_file) as data_file: 
    synapse = json.load(data_file) 
    synapse_0 = np.asarray(synapse['synapse0']) 
    synapse_1 = np.asarray(synapse['synapse1'])

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

Training with 25 neurons, alpha:0.01 
Input matrix: 405x651    Output matrix: 1x2
delta after 10000 iterations:0.00887689718091
delta after 20000 iterations:0.00343929900577
delta after 30000 iterations:0.00261931654725
delta after 40000 iterations:0.00219676206446
delta after 50000 iterations:0.00192663965152
delta after 60000 iterations:0.00173472807108
delta after 70000 iterations:0.00158934630629
delta after 80000 iterations:0.00147431934455
delta after 90000 iterations:0.00138039499209
delta after 100000 iterations:0.00130184300606
saved synapses to: synapses.json
processing time: 548.1557412147522 seconds


In [257]:
accuracy = 0

for i in range(len(test)):
    print(test_names[i])
    result = classify(test[i])
    
    if ('violation' in test_names[i] and result[0][0] == 'violation') or \
     ('violation' not in test_names[i] and result[0][0] == 'notviolation'):
        accuracy += 1.0
    
    print() 
    
 

test/733e522cba6c8d3fc79cd32546fadf54.py
classification: [['notviolation', 0.99988149683810013]]

test/04aa2288beeb67289e9d5edaa9c10d41.py
classification: [['notviolation', 0.97996282279966085]]

test/57327a51d2fb7ab381497f8420b71411.py
classification: [['notviolation', 0.9794532133536501]]

test/82f2ac914f72771bbfe68ab70ab8c0d1.py
classification: [['notviolation', 0.99968959741189811]]

test/3757d0261bdeddbc4b79aa52169b7bce.py
classification: [['notviolation', 0.94392288357704845]]

test/1927ee1c31d541abeae56ec1a2e0fadf.py
classification: [['notviolation', 0.99901984844366587]]

test/e72f92da1eaa50345ffea0899f9ebbe5.py
classification: [['notviolation', 0.99999917863605792]]

test/de97abfb65d2b5327965ab99089f56e4.py
classification: [['notviolation', 0.99988575270400248]]

test/violation_46.py
classification: [['violation', 0.99986744066675404]]

test/violation_33.py
classification: [['violation', 0.99736829923059334]]

test/90036ab09a7539f7dff743e33c6e24cf.py
classification: [['notviol

In [264]:
print("Accuracy: %s%%" %(accuracy/len(test) * 100))   

Accuracy: 90.12345679012346%


# 6) Resultados

Considerando os números apresentados acima podemos concluir que o melhor resultado foi obtido com a configuração 3 na qual existe uma maior quantidade de neurônios e o alpha, que são os passos do gradiente, é menor, o que se ajusta com a quantidade de iterações. Esta última configuração conseguiu classificar <b>91,35%</b> dos casos de testes, o que é impressionante dado que a quantidade de dados de treino para violações é a metade das não-violações.

Entretanto algumas ameaças a validade devem ser ressaltadas, como o dataset reduzido por ser somente de uma unidade da disciplina, e que como só se tem duas camadas na rede a medida que se aumentam as iterações e o número de neurônios o tempo de treino aumenta consideravelmente e em alguns cenários de configuração se tornou impraticável para demonstração.

# Demo

In [256]:
demo = get_data_from_files('demo')
demo_file = get_files_names('demo')

for i in range(len(demo)):
    print(clean_up_sentence(demo[i]))
    print()
    print(demo_file[i])
    classify(demo[i])

['def', 'demo_function', 'list', ':', 'cont', '=', '0', 'for', 'i', 'in', 'rang', 'len', 'list', ':', 'if', 'list', '[', 'i', ']', '==', "'e", "'", ':', 'cont', '+=', '1', 'return', 'cont']

demo/demo_file_program.py
classification: [['notviolation', 0.8332291482694385], ['violation', 0.20242930339405568]]
